# Student RAG Project - Guided Implementation

## Welcome!

In this project, you'll build a **RAG (Retrieval-Augmented Generation)** system that can answer questions about your documents.

### What You'll Learn:
- ✅ File I/O (reading documents)
- ✅ String manipulation (text chunking)
- ✅ Functions and parameters
- ✅ Lists and dictionaries
- ✅ Loops and conditionals
- ✅ Basic calculations and statistics

### What's Provided for You:
- ✅ Embedding model (converts text to numbers)
- ✅ Vector database (stores and searches embeddings)
- ✅ LLM connection (generates answers)

### Your Tasks:
You'll complete **TODO sections** marked with `# TODO:` comments.

Let's get started! 🚀

---
## Setup: Import Libraries

In [1]:
# Import the pre-built helper module
from rag_helpers import (
    EmbeddingModel,
    VectorDatabase,
    LLM,
    Timer,
    print_separator,
    print_search_results,
    print_rag_answer,
    check_setup
)

# Import standard Python libraries you'll use
from pathlib import Path
from typing import List, Dict
import json

# Check if everything is installed correctly
check_setup()

Checking setup...
✓ chromadb is installed
✓ sentence_transformers is installed
✓ requests is installed

✓ All required packages are installed!
You're ready to start!


True

---
## Configuration

Set up the basic settings for your RAG system.

In [3]:
# TODO: Change this to point to YOUR documents folder
DOCS_FOLDER = "./docs/text"

# Chunking settings (you can experiment with these!)
CHUNK_SIZE = 500      # How many characters per chunk
OVERLAP = 50          # How many characters overlap between chunks

# How many results to retrieve for each query
TOP_K = 3

print(f"Configuration:")
print(f"  Documents folder: {DOCS_FOLDER}")
print(f"  Chunk size: {CHUNK_SIZE} characters")
print(f"  Overlap: {OVERLAP} characters")
print(f"  Top-K results: {TOP_K}")

Configuration:
  Documents folder: ./docs/text
  Chunk size: 500 characters
  Overlap: 50 characters
  Top-K results: 3


---
## TODO #1: Document Loading

**Your Task:** Write a function to load all text files from a folder.

**What to do:**
1. Loop through all `.txt` files in the folder
2. Read each file's content
3. Store the content and filename in a dictionary
4. Return a list of these dictionaries

**Python concepts:** File I/O, loops, dictionaries, lists

In [4]:
from pathlib import Path
from typing import List, Dict

def load_documents(folder_path: str) -> List[Dict[str, str]]:
    """
    Load all text documents from a folder.

    Args:
        folder_path: Path to folder containing .txt files

    Returns:
        List of dictionaries, each containing:
        - 'content': the text content of the file
        - 'filename': the name of the file
    """
    documents = []  # Start with empty list
    folder = Path(folder_path)

    # Loop through all .txt files in the folder
    for file_path in folder.glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        doc_info = {
            "content": content,
            "filename": file_path.name
        }
        documents.append(doc_info)

    print(f"✓ Loaded {len(documents)} documents")
    return documents


In [5]:
# 🛠 Debug + Load Documents Cell

import os
from pathlib import Path

print("Working directory:", os.getcwd())

# Check some common candidate paths
candidate_paths = ["docs/text", "./docs/text", "docs", "./docs"]
print("\nChecking candidate paths for .txt files:")
for cp in candidate_paths:
    folder = Path(cp)
    print(f"\nPath: {cp}")
    print("  Exists?:", folder.exists())
    if folder.exists():
        txt_files = list(folder.glob("*.txt"))
        print("  .txt files found:", txt_files)
    else:
        print("  .txt files found: []")

# Try to auto-select a good DOCS_FOLDER based on what we just saw
if Path("docs/text").exists() and list(Path("docs/text").glob("*.txt")):
    DOCS_FOLDER = "docs/text"
elif Path("./docs/text").exists() and list(Path("./docs/text").glob("*.txt")):
    DOCS_FOLDER = "./docs/text"
elif Path("docs").exists() and list(Path("docs").glob("*.txt")):
    DOCS_FOLDER = "docs"
elif Path("./docs").exists() and list(Path("./docs").glob("*.txt")):
    DOCS_FOLDER = "./docs"

print(f"\nUsing DOCS_FOLDER = {DOCS_FOLDER!r}")

# Now call your existing load_documents() with the detected folder
documents = load_documents(DOCS_FOLDER)

# Display first document (if any were loaded)
if documents:
    print(f"\nFirst document: {documents[0]['filename']}")
    print(f"Content preview: {documents[0]['content'][:200]}...")
else:
    print("⚠️  No documents loaded! Check your folder path again.")


Working directory: c:\Users\Mejia\Dev\Rag\IS640_rag_project

Checking candidate paths for .txt files:

Path: docs/text
  Exists?: True
  .txt files found: [WindowsPath('docs/text/arrozConGandules.txt'), WindowsPath('docs/text/coquito.txt'), WindowsPath('docs/text/empanadilla.txt'), WindowsPath('docs/text/panDeAgua.txt'), WindowsPath('docs/text/pastelon.txt'), WindowsPath('docs/text/pernil.txt'), WindowsPath('docs/text/sample1.txt'), WindowsPath('docs/text/sofrito.txt')]

Path: ./docs/text
  Exists?: True
  .txt files found: [WindowsPath('docs/text/arrozConGandules.txt'), WindowsPath('docs/text/coquito.txt'), WindowsPath('docs/text/empanadilla.txt'), WindowsPath('docs/text/panDeAgua.txt'), WindowsPath('docs/text/pastelon.txt'), WindowsPath('docs/text/pernil.txt'), WindowsPath('docs/text/sample1.txt'), WindowsPath('docs/text/sofrito.txt')]

Path: docs
  Exists?: True
  .txt files found: []

Path: ./docs
  Exists?: True
  .txt files found: []

Using DOCS_FOLDER = 'docs/text'
✓ Loaded 8 do

In [6]:
documents = load_documents(DOCS_FOLDER)


✓ Loaded 8 documents


---
## TODO #2: Text Chunking Function

**Your Task:** Write a function to split long text into smaller chunks with overlap.

**Why?** Long documents are too big for embeddings. We need to split them into smaller pieces.

**What to do:**
1. Start at the beginning of the text
2. Take a chunk of `chunk_size` characters
3. Move forward by `chunk_size - overlap` characters
4. Repeat until you reach the end

**Python concepts:** String slicing, loops, lists

In [7]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """
    Split text into overlapping chunks.
    """
    chunks = []
    position = 0

    # Safety: ensure overlap is smaller than chunk size
    step = chunk_size - overlap
    if step <= 0:
        raise ValueError("Overlap must be smaller than chunk_size")

    # Main loop
    while position < len(text):
        chunk = text[position : position + chunk_size]

        # Only add non-empty chunks
        if chunk.strip():
            chunks.append(chunk)

        # Move forward
        position += step

    return chunks





In [8]:
# Test your chunking function
test_text = "This is a test. " * 50  # Create a long test string
test_chunks = chunk_text(test_text, chunk_size=100, overlap=20)

print(f"Test text length: {len(test_text)} characters")
print(f"Number of chunks: {len(test_chunks)}")
print(f"\nFirst chunk: {test_chunks[0]}")
if len(test_chunks) > 1:
    print(f"Second chunk: {test_chunks[1]}")


Test text length: 800 characters
Number of chunks: 10

First chunk: This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This
Second chunk: This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This


---
## TODO #3: Process All Documents into Chunks

**Your Task:** Use your chunking function to split ALL documents into chunks and create metadata.

**What to do:**
1. Loop through each document
2. Chunk the document's content
3. For each chunk, create metadata (which file it came from, which chunk number)
4. Store everything in a list

**Python concepts:** Nested loops, dictionaries, enumerate

In [9]:
def process_documents(documents: List[Dict[str, str]],
                     chunk_size: int = 500,
                     overlap: int = 50) -> tuple:
    """
    Process all documents into chunks with metadata.
    """
    chunk_texts = []
    chunk_metadatas = []

    for doc in documents:
        # Get document content and filename
        content = doc["content"]
        filename = doc["filename"]

        # Chunk the document using your chunk_text function
        chunks = chunk_text(content, chunk_size=chunk_size, overlap=overlap)

        # For each chunk:
        for idx, chunk in enumerate(chunks):
            chunk_texts.append(chunk)

            metadata = {
                "source": filename,
                "chunk_id": idx
            }
            chunk_metadatas.append(metadata)

    print(f"✓ Created {len(chunk_texts)} chunks from {len(documents)} documents")
    return chunk_texts, chunk_metadatas


In [10]:
chunk_texts, chunk_metadatas = process_documents(documents, CHUNK_SIZE, OVERLAP)

if chunk_texts:
    print(f"\nExample chunk:")
    print(f"  Source: {chunk_metadatas[0]['source']}")
    print(f"  Chunk ID: {chunk_metadatas[0]['chunk_id']}")
    print(f"  Text: {chunk_texts[0][:200]}...")


✓ Created 148 chunks from 8 documents

Example chunk:
  Source: arrozConGandules.txt
  Chunk ID: 0
  Text: Arroz Con Gandules (Puerto Rican Rice with Pigeon Peas)
Jannese
48–61 minutes

December 4, 2022	

There’s nothing more quintessentially Puerto Rican than arroz con gandules.  

This post may contain a...


---
## Pre-Built: Create Embeddings and Store in Database

This part uses the pre-built helpers. Just run these cells - no coding needed! ✨

In [11]:


# Initialize the embedding model (pre-built)
print("Initializing embedding model...")
embedder = EmbeddingModel()

# Create embeddings for all chunks (pre-built)
print("\nCreating embeddings...")
embeddings = embedder.embed_multiple(chunk_texts)
print(f"✓ Created {len(embeddings)} embeddings")



Initializing embedding model...
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
✓ Model loaded!

Creating embeddings...
Embedding 148 texts...
✓ Complete!
✓ Created 148 embeddings


In [12]:
# Initialize vector database (pre-built)
print("Initializing vector database...")
vector_db = VectorDatabase()

# Add chunks to database (pre-built)
print("\nAdding chunks to database...")
vector_db.add_chunks(chunk_texts, embeddings, chunk_metadatas)

Initializing vector database...
✓ Vector database initialized
  Collection: student_rag
  Current documents: 252

Adding chunks to database...
✓ Added 148 chunks to database


In [13]:
# Initialize LLM connection (pre-built)
print("Connecting to Ollama LLM...")
llm = LLM()

# Test the connection
print("\nTesting LLM connection...")
if llm.test_connection():
    print("✓ LLM is working!")
else:
    print("⚠️  LLM connection failed! Make sure Docker container is running.")

Connecting to Ollama LLM...
✓ LLM initialized: mistral:7b at http://127.0.0.1:11434

Testing LLM connection...
✓ LLM is working!


---
## TODO #4: RAG Query Function

**Your Task:** Write the main RAG function that ties everything together!

**What to do:**
1. Embed the user's question
2. Search the database for similar chunks
3. Build a prompt with the retrieved context
4. Ask the LLM to answer based on the context
5. Return the answer and metadata

**Python concepts:** Functions, string formatting, dictionaries

In [14]:
def rag_query(question: str, top_k: int = 3) -> Dict:
    """
    Answer a question using RAG (Retrieval-Augmented Generation).
    """

    # Start timer
    timer = Timer()
    timer.start()

    # Step 1: Embed question
    query_embedding = embedder.embed_text(question)

    # Step 2: Search database
    results = vector_db.search(query_embedding, top_k=top_k)

    # Step 3: Extract results
    # results["documents"] and results["metadatas"] are lists of lists
    retrieved_chunks = results["documents"][0] if results["documents"] else []
    retrieved_metadata = results["metadatas"][0] if results["metadatas"] else []

    # Step 4: Build context
    context = "\n\n".join(retrieved_chunks) if retrieved_chunks else ""

    # Step 5: Create prompt (already provided)
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {question}

Answer:"""

    # Step 6: Generate answer
    answer = llm.generate_answer(prompt)

    # Step 7: Extract sources
    sources = [meta["source"] for meta in retrieved_metadata] if retrieved_metadata else []

    # Stop timer
    elapsed_time = timer.stop()

    # Step 8: Return results
    return {
        "question": question,
        "answer": answer,
        "sources": sources,
        "contexts": retrieved_chunks,
        "time": elapsed_time
    }


---
## Test Your RAG System!

Let's try asking some questions!

In [15]:
# Test question 1
result = rag_query("What can I make with Chicken and Rice")

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

======================== RAG ANSWER ========================

QUESTION: What can I make with Chicken and Rice

ANSWER:
 Arroz Con Gandules (Puerto Rican Rice with Pigeon Peas) is a dish that includes chicken, rice, pigeon peas, adobo seasoning blend, sazón flavoring blend, cilantro, and olives. You can find the recipe in the provided context.

SOURCES: arrozConGandules.txt, pastelon.txt
TIME: 74.24 seconds


In [16]:
# Try your own question!
my_question = "How do you bake Pan de Agua?"  # Change this!

result = rag_query(my_question)
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

======================== RAG ANSWER ========================

QUESTION: How do you bake Pan de Agua?

ANSWER:
 Pan de agua is baked by placing the proofed dough in a cold oven set above a pan of boiling water. The oven temperature is then raised and the bread is baked until golden brown.

SOURCES: panDeAgua.txt
TIME: 72.27 seconds


---
## TODO #5: Create Test Dataset

**Your Task:** Create a list of test questions to evaluate your RAG system.

**What to do:**
1. Think of 10 questions your documents can answer
2. For each question, write the expected answer
3. Store them in a structured format

**Python concepts:** Lists, dictionaries, data structures

In [17]:
# TODO: Create your test dataset!
# Creating 10 recipe-related test questions based on the documents in docs/text

test_questions = [
    {
        'question': 'What are the main ingredients in arroz con gandules?',
        'expected_answer': 'rice, pigeon peas (gandules), sofrito, sazón, tomato sauce, olives, and pork or ham',
        'category': 'factual'
    },
    {
        'question': 'How long should pernil be marinated before cooking?',
        'expected_answer': 'at least several hours, but preferably overnight or up to 24 hours',
        'category': 'factual'
    },
    {
        'question': 'At what temperature and for how long do you roast pernil?',
        'expected_answer': 'several hours at around 300–350°F until tender and the skin gets crispy',
        'category': 'procedural'
    },
    {
        'question': 'What is sofrito used for in Puerto Rican cooking?',
        'expected_answer': 'a flavor base used to season dishes like arroz con gandules, stews, and beans',
        'category': 'factual'
    },
    {
        'question': 'How do you assemble a pastelón?',
        'expected_answer': 'layer fried sweet plantains with seasoned ground beef and cheese, then bake',
        'category': 'procedural'
    },
    {
        'question': 'What ingredients are traditionally used to make coquito?',
        'expected_answer': 'coconut milk, cream of coconut, rum, spices, sweetened condensed milk, and vanilla',
        'category': 'factual'
    },
    {
        'question': 'What fillings are commonly used in empanadillas?',
        'expected_answer': 'seasoned ground beef, chicken, cheese, or other savory fillings',
        'category': 'factual'
    },
    {
        'question': 'What ingredients are blended to make homemade sofrito?',
        'expected_answer': 'peppers, onions, garlic, cilantro, recao, and other herbs',
        'category': 'factual'
    },
    {
        'question': 'Which recipes from this collection are commonly served during holidays?',
        'expected_answer': 'pernil, arroz con gandules, pastelón, and coquito',
        'category': 'inferential'
    },
    {
        'question': 'How should leftover arroz con gandules be reheated?',
        'expected_answer': 'reheat gently with a little water on the stove or microwaved while covered',
        'category': 'procedural'
    }
]

print(f"✓ Created {len(test_questions)} test questions")
print(f"\nExample question:")
print(f"  Q: {test_questions[0]['question']}")
print(f"  Expected: {test_questions[0]['expected_answer']}")


✓ Created 10 test questions

Example question:
  Q: What are the main ingredients in arroz con gandules?
  Expected: rice, pigeon peas (gandules), sofrito, sazón, tomato sauce, olives, and pork or ham


---
## TODO #6: Calculate Evaluation Metrics

**Your Task:** Write functions to measure how well your RAG system performs.

**Python concepts:** Functions, calculations, statistics

In [18]:
def calculate_average_latency(results: List[Dict]) -> float:
    """
    Calculate average response time.

    Args:
        results: List of result dictionaries (each has 'time' field)

    Returns:
        Average time in seconds
    """
    if not results:
        return 0.0

    # Extract all 'time' values and sum them
    total_time = sum(result.get('time', 0) for result in results)

    # Divide by number of results
    average_time = total_time / len(results)

    return average_time


def count_successful_retrievals(results: List[Dict]) -> int:
    """
    Count how many queries successfully retrieved context.

    Args:
        results: List of result dictionaries

    Returns:
        Number of successful retrievals
    """
    count = 0

    for result in results:
        # 'contexts' should be a non-empty list
        if result.get('contexts'):
            count += 1

    return count


def get_all_sources(results: List[Dict]) -> List[str]:
    """
    Get unique list of all sources used.

    Args:
        results: List of result dictionaries

    Returns:
        List of unique source filenames
    """
    all_sources = set()

    for result in results:
        for source in result.get('sources', []):
            all_sources.add(source)

    return list(all_sources)


print("✓ Evaluation functions defined!")


✓ Evaluation functions defined!


---
## TODO #7: Run Complete Evaluation

**Your Task:** Test your RAG system with all test questions and calculate metrics.

**Python concepts:** Loops, function calls, data aggregation

In [ ]:
def run_evaluation(test_questions: List[Dict]) -> List[Dict]:
    """
    Run RAG system on all test questions.

    Args:
        test_questions: List of test question dictionaries

    Returns:
        List of result dictionaries
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create empty list for results
    # 2. For each test question:
    #    - Get the question text
    #    - Call rag_query() with the question
    #    - Add result to results list
    # 3. Return results

    results = []

    # Your code here:
    for test in test_questions:
        # Get question
        # Run RAG query
        # Store result
        pass  # Remove this 'pass' and write your code

    return results


# Run evaluation on all test questions
print("Running evaluation on all test questions...\n")
all_results = run_evaluation(test_questions)

print(f"\n✓ Completed {len(all_results)} tests")

---
## Display Results

Show the evaluation metrics and results.

In [ ]:
# Calculate metrics using your functions
avg_latency = calculate_average_latency(all_results)
successful_retrievals = count_successful_retrievals(all_results)
all_sources_used = get_all_sources(all_results)
hit_rate = successful_retrievals / len(all_results) if all_results else 0

# Display metrics
print_separator("EVALUATION RESULTS")
print(f"\nTotal Questions Tested: {len(all_results)}")
print(f"Successful Retrievals: {successful_retrievals}")
print(f"Hit Rate: {hit_rate:.2%}")
print(f"Average Latency: {avg_latency:.2f} seconds")
print(f"\nSources Used: {', '.join(all_sources_used)}")
print_separator()

In [ ]:
# Display individual results
print("\nIndividual Test Results:\n")

for i, result in enumerate(all_results, 1):
    print(f"[Test {i}]")
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer'][:200]}...")
    print(f"Sources: {', '.join(set(result['sources']))}")
    print(f"Time: {result['time']:.2f}s")
    print("-" * 60)
    print()

---
## Save Your Results

Save your test results to a JSON file for your report.

In [ ]:
# Save results to JSON file
results_summary = {
    'metrics': {
        'total_questions': len(all_results),
        'successful_retrievals': successful_retrievals,
        'hit_rate': hit_rate,
        'average_latency': avg_latency
    },
    'results': all_results
}

with open('evaluation_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("✓ Results saved to 'evaluation_results.json'")

---
## Congratulations! 🎉

You've successfully built a RAG system!

### What You Accomplished:
✅ Loaded documents from files  
✅ Chunked text with overlap  
✅ Created a RAG query pipeline  
✅ Built a test dataset  
✅ Calculated evaluation metrics  
✅ Generated a results report  

### Next Steps:
- Try different chunk sizes and overlaps
- Add more test questions
- Experiment with different values for `top_k`
- Analyze which questions work best
- Write up your findings in a report

### For Your Report:
1. Describe your document collection
2. Explain your chunking strategy
3. Present your evaluation metrics
4. Show examples of good and bad answers
5. Discuss what you learned

Great job! 🚀